In [8]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from urllib.parse import quote_plus

In [9]:
# Load environment variables from .env
load_dotenv()

True

In [10]:
# Fetch database credentials from .env
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

# Encode the password to handle special characters
password_encoded = quote_plus(DB_PASSWORD)

# Use only this connection string (DO NOT create another one)
connection_url = f'postgresql://{DB_USER}:{password_encoded}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# Use this engine throughout the script
engine = create_engine(connection_url)

In [11]:
# Establish a connection using psycopg2
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
cursor = conn.cursor()

print("Connection to PostgreSQL successful!")

Connection to PostgreSQL successful!


In [15]:
# Calculating the customers percentage most likely to churn
query = """
SELECT 
    ROUND( (SUM(CASE WHEN churn_probability >= 50 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) AS churn_rate
FROM churn_data;
"""

df_customer_churn = pd.read_sql(query, engine)

print(df_customer_churn)

   churn_rate
0         0.0


In [21]:
query = "SELECT MIN(churn_probability), MAX(churn_probability) FROM churn_data;"
df_range = pd.read_sql(query, engine)
print(df_range)

   min  max
0    0    1


In [22]:
query = "SELECT DISTINCT churn_probability FROM churn_data ORDER BY churn_probability DESC LIMIT 10;"
df_check = pd.read_sql(query, engine)
print(df_check)

   churn_probability
0                  1
1                  0


In [23]:
query = """
SELECT 
    ROUND( (SUM(CASE WHEN churn_probability = 1 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) AS churn_rate
FROM churn_data;
"""

df_customer_churn = pd.read_sql(query, engine)
print(df_customer_churn)

   churn_rate
0       50.24


In [24]:
# Checking churn rate by purchase behavior
query = """
SELECT 
    purchase_behavior,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN churn_probability = 1 THEN 1 ELSE 0 END) AS churned_customers,
    ROUND((SUM(CASE WHEN churn_probability = 1 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) AS churn_rate
FROM churn_data
GROUP BY purchase_behavior
ORDER BY churn_rate DESC;
"""

df_purchase_behavior = pd.read_sql(query, engine)
print(df_purchase_behavior)

    purchase_behavior  total_customers  churned_customers  churn_rate
0                 9.0              427                233       54.57
1                15.0              410                221       53.90
2                19.0              438                230       52.51
3                17.0              411                215       52.31
4                11.0              401                208       51.87
5                16.0              412                213       51.70
6                12.0              390                199       51.03
7                10.0              439                224       51.03
8                 8.0              431                219       50.81
9                18.0              430                218       50.70
10               13.0              415                208       50.12
11                1.0              462                231       50.00
12               14.0              426                209       49.06
13                7.

In [26]:
# Checking churn rate by spending category
query = """
SELECT 
    CASE 
        WHEN average_order_value < 50 THEN 'Low Spend (< $50)'
        WHEN average_order_value BETWEEN 50 AND 200 THEN 'Medium Spend ($50-$200)'
        ELSE 'High Spend (> $200)'
    END AS spending_category,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN churn_probability = 1 THEN 1 ELSE 0 END) AS churned_customers,
    ROUND((SUM(CASE WHEN churn_probability = 1 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) AS churn_rate
FROM churn_data
GROUP BY spending_category
ORDER BY churn_rate DESC;
"""

df_aov_churn = pd.read_sql(query, engine)
print(df_aov_churn)

         spending_category  total_customers  churned_customers  churn_rate
0  Medium Spend ($50-$200)             6639               3342       50.34
1        Low Spend (< $50)             1361                677       49.74


In [27]:
# Checking churn rate by Purchase Frequency
query = """
SELECT 
    CASE 
        WHEN time_between_purchases_log < 1 THEN 'Very Frequent (< 1 day)'
        WHEN time_between_purchases_log BETWEEN 1 AND 7 THEN 'Frequent (1-7 days)'
        WHEN time_between_purchases_log BETWEEN 7 AND 30 THEN 'Occasional (7-30 days)'
        ELSE 'Rare (> 30 days)'
    END AS purchase_frequency_category,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN churn_probability = 1 THEN 1 ELSE 0 END) AS churned_customers,
    ROUND((SUM(CASE WHEN churn_probability = 1 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) AS churn_rate
FROM churn_data
GROUP BY purchase_frequency_category
ORDER BY churn_rate DESC;
"""

df_purchase_freq = pd.read_sql(query, engine)
print(df_purchase_freq)

  purchase_frequency_category  total_customers  churned_customers  churn_rate
0         Frequent (1-7 days)              287                152       52.96
1      Occasional (7-30 days)             2211               1116       50.47
2            Rare (> 30 days)             5502               2751       50.00


In [28]:
# Define the path to save CSV files
data_folder = "../data/"
 
# Export each dataframe to CSV
df_customer_churn.to_csv(data_folder + "churn_overall.csv", index=False)
df_purchase_behavior.to_csv(data_folder + "churn_by_purchase_behavior.csv", index=False)
df_aov_churn.to_csv(data_folder + "churn_by_spending.csv", index=False)
df_purchase_freq.to_csv(data_folder + "churn_by_purchase_frequency.csv", index=False)

print("Churn insights saved to CSV successfully!")

Churn insights saved to CSV successfully!
